In [ ]:
from decoder import Decoder
from decoderattn import Attention_Decoder
from encoder import Encoder
from evaluatemodel import MySeq2SeqModelEvaluation
from train_with_batch import Seq2SeqTrainer
from trainmodel import Seq2SeqTrainer
from data_proc import *

In [15]:
import torch
from io import open
import torch

use_cuda = torch.cuda.is_available()

In [16]:
start_token = 0
end_token = 1
max_length = 50
num_epochs = 75000

In [30]:
obj= DataPreparation('fra', 'eng', True)
input_lang, output_lang, pairs= obj.prepare_data(reverse=True)


Reading lines...
Form:  ['l or a . dollars l once ?', ' gold ?']
Read 2000 sentence pairs
Trimmed to 1837 sentence pairs
Counting words...
Counted words:
eng 6924
fra 6019


In [18]:
hidden_size = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
attn_decoder = Attention_Decoder(hidden_size, output_lang.n_words, max_length , dropoutP=0.1).to(device)

encoder = Encoder(input_size = input_lang.n_words, hidden_size = hidden_size, device = device).to(device)



In [ ]:
TRAIN1=Seq2SeqTrainer(encoder, attn_decoder, max_length, start_token, end_token,input_lang ,output_lang , device)
losses=TRAIN1.custom_training_function(num_epochs, pairs, 5000, plot_interval=50, learning_rate=0.01)

: 

In [ ]:
# sauvegarde du modèle
torch.save(encoder,"encoder_bi.pt")
torch.save(attn_decoder,"attn_decoder_bi.pt")

In [ ]:
# chargement du modèle
encoder_loaded = torch.load("encoder_bi.pt")
attn_decoder_loaded = torch.load("attn_decoder_bi.pt")

In [ ]:
modele_evaluation=MySeq2SeqModelEvaluation(encoder_loaded, attn_decoder_loaded,  max_length, input_lang, output_lang, start_token, end_token, device)
modele_evaluation.evaluateRandomly(pairs)

In [ ]:
output_words, attentions = modele_evaluation.forward("j'ai trop froid .")

In [ ]:
print(output_words)